In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(42)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
df=pd.read_csv('fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.shape

(60000, 785)

In [ ]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train=X_train/255.0
X_test=X_test/255.0

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,x,y):
        self.x=torch.tensor(x,dtype=torch.float32).reshape(-1,1,28,28)
        self.y=torch.tensor(y,dtype=torch.long)

    def __len__(self):
        return len(self.x)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

In [ ]:
train_dataset=CustomDataset(X_train,y_train)
test_dataset=CustomDataset(X_test,y_test)

In [ ]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [ ]:
class MyNN(nn.Module):
  def __init__(self,input_fetaures):
    super().__init__()

    self.fetaures=nn.Sequential(
        nn.Conv2d(input_fetaures,32,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.Conv2d(32,64,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2),
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(64,10)
    )

  def forward(self,x):
    x=self.fetaures(x)
    x=self.classifier(x)

    return x

In [ ]:
learning_rate=0.01
epochs=100

In [ ]:
model=MyNN(1)

model.to(device)

criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-4)


In [ ]:
#training loop

for epoch in range(epochs):
    total_epoch_loss=0
    for batch_fetaures,batch_labels in train_loader:

        batch_features = batch_fetaures.to(device=device)
        batch_labels = batch_labels.to(device=device)

        output=model(batch_features)
        loss=criterion(output,batch_labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_epoch_loss =total_epoch_loss+loss.item()

    avg_loss=total_epoch_loss/len(train_loader)
    print(f'Epoch: {epoch+1}, Loss: {avg_loss:.4f}')


Epoch: 1, Loss: 0.6474
Epoch: 2, Loss: 0.3856
Epoch: 3, Loss: 0.3284
Epoch: 4, Loss: 0.2940
Epoch: 5, Loss: 0.2641
Epoch: 6, Loss: 0.2457
Epoch: 7, Loss: 0.2267
Epoch: 8, Loss: 0.2122
Epoch: 9, Loss: 0.2016
Epoch: 10, Loss: 0.1899
Epoch: 11, Loss: 0.1779
Epoch: 12, Loss: 0.1664
Epoch: 13, Loss: 0.1552
Epoch: 14, Loss: 0.1516
Epoch: 15, Loss: 0.1441
Epoch: 16, Loss: 0.1351
Epoch: 17, Loss: 0.1282
Epoch: 18, Loss: 0.1204
Epoch: 19, Loss: 0.1200
Epoch: 20, Loss: 0.1098
Epoch: 21, Loss: 0.1041
Epoch: 22, Loss: 0.0957
Epoch: 23, Loss: 0.0927
Epoch: 24, Loss: 0.0886
Epoch: 25, Loss: 0.0866
Epoch: 26, Loss: 0.0820
Epoch: 27, Loss: 0.0791
Epoch: 28, Loss: 0.0755
Epoch: 29, Loss: 0.0704
Epoch: 30, Loss: 0.0678
Epoch: 31, Loss: 0.0657
Epoch: 32, Loss: 0.0610
Epoch: 33, Loss: 0.0601
Epoch: 34, Loss: 0.0583
Epoch: 35, Loss: 0.0599
Epoch: 36, Loss: 0.0552
Epoch: 37, Loss: 0.0538
Epoch: 38, Loss: 0.0489
Epoch: 39, Loss: 0.0473
Epoch: 40, Loss: 0.0469
Epoch: 41, Loss: 0.0454
Epoch: 42, Loss: 0.0415
E

In [ ]:
model.eval()

MyNN(
  (fetaures): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
total=0
correct=0

with torch.no_grad():
    for batch_fetaures,batch_labels in test_loader:

        batch_features = batch_fetaures.to(device=device)
        batch_labels = batch_labels.to(device=device)

        output=model(batch_features)
        _,predicted=torch.max(output,1)

        total+=batch_labels.size(0)
        correct+=(predicted==batch_labels).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 92.68333333333334 %
